In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

cnx = sqlite3.connect('database.sqlite')

player_data = pd.read_sql("SELECT id, player_api_id, player_name, birthday, height, weight FROM Player", cnx)
player_stats_data = pd.read_sql("SELECT id, player_api_id, date, overall_rating, potential, preferred_foot, attacking_work_rate, defensive_work_rate, crossing, finishing, heading_accuracy, short_passing, volleys, dribbling, curve, free_kick_accuracy, long_passing, ball_control, acceleration, sprint_speed, agility, reactions, shot_power, jumping, stamina, strength, long_shots, aggression, interceptions, positioning, vision, penalties, marking, standing_tackle, sliding_tackle, gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes FROM Player_Attributes", cnx)
match_data = pd.read_sql("SELECT id, country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11, goal, shoton, shotoff, foulcommit, card, cross, corner, possession, B365H, B365D, B365A FROM Match", cnx)
league_data = pd.read_sql("SELECT id, country_id, name FROM League", cnx)
country_data = pd.read_sql("SELECT id, name FROM Country", cnx)
team_data = pd.read_sql("SELECT id, team_api_id, team_long_name, team_short_name FROM Team", cnx)
team_attributes_data = pd.read_sql("SELECT id, team_api_id, date, buildUpPlaySpeed, buildUpPlaySpeedClass, buildUpPlayDribbling, buildUpPlayDribblingClass, buildUpPlayPassing, buildUpPlayPassingClass, buildUpPlayPositioningClass, chanceCreationPassing, chanceCreationPassingClass, chanceCreationCrossing, chanceCreationCrossingClass, chanceCreationShooting, chanceCreationShooting, chanceCreationPositioningClass, defencePressure, defencePressureClass, defenceAggression, defenceAggressionClass, defenceTeamWidth, defenceTeamWidthClass, defenceDefenderLineClass FROM Team_Attributes", cnx)


def null_counts( dataframe ):
    '''
    Get percentage of empty values per column

    Args:
        dataframe: dataframe containing at least one column and value

    Returns:
        Returns nothing, prints percentages
    '''
    null_counts = dataframe.isnull().sum()
    null_percents = null_counts / len(dataframe) * 100
    print(null_percents)


null_counts(player_data)
null_counts(player_stats_data)
null_counts(match_data)
null_counts(league_data)
null_counts(country_data)
null_counts(team_data)
null_counts(team_attributes_data)

# Datenaufbereitung
In diesem Code-Abschnitt werden die Daten bereinigt und für das maschinelle Lernen aufbereitet. Dazu werden die im vorherigen Abschnitt gesammelten Erkenntnisse zu den Anteilen von Leeren Werten je Spalte dazu genutzt, Spalten mit mangelnder Aussagekraft vollständig zu entfernen (`drop()`). Daraufhin wird sich außerdem der verbliebenen leeren Werte in allen Zeilen angenommen.

## Spielerratings
Die Tabelle Player_Attributes (Dtaframe `player_stats_data`) enthält Spiellerratings für alle Spieler zu verschiedenen Zeitpunkten. Dabei haben manche Spieler mehrere Ratings pro Jahr, andere nur ein Rating pro Jahr, wieder andere nur ein Rating in mehreren Jahren. Um diese unterschiedlichen Vorraussetzungen für alle Spieler auf ein Niveau zu bringen, werden mehrere unterjährige Ratings auf ein einzelnes gemittelt. Im folgenden wird dann für jeden Spieler das Rating aus dem benötigten Jahr verwendet. Gibt es kein Rating aus diesem Jahr, wird das naheliegenste Rating herangezogen.

In [ ]:
match_data = match_data.drop(columns=["goal", "shoton", "shotoff", "foulcommit", "card", "cross", "corner", "possession"], axis=1)
team_attributes_data = team_attributes_data.drop("buildUpPlayDribbling", axis=1)

df_dropped = match_data.dropna()
player_stats_data = player_stats_data.dropna()

In [ ]:


def get_nearest_rating(date, player_api_id):
    '''
    Get player rating closest to the provided date

    Args:
        date: Date as int. (yyyymmdd)
        player_api_id: player_api_id as int.

    Returns:
        Returns a new dataframe containing all player attributes for the nearest date.

    Raises:
        ValueError: No data for provided player_api_id
    '''
    all_ratings_for_player: pd.DataFrame = player_stats_data.loc[player_stats_data["player_api_id"] == player_api_id]
    if (all_ratings_for_player.empty):
        raise ValueError(f"No data for provided player_api_id: {player_api_id}")
    if (len(all_ratings_for_player.index) == 1):
        return all_ratings_for_player
    intdate: pd.DataFrame = pd.DataFrame(columns=["date"], dtype=np.int8)
    for index, row in all_ratings_for_player.iterrows():
        rowdate: str = row["date"][:10]
        rowdate = rowdate.replace('-', '')
        intdate.loc[index] = int(rowdate)
    intdate = intdate.set_index('date')
    intdate.sort_index(inplace = True)
    index = intdate.index.get_indexer([date], method="nearest")
    all_ratings_for_player.reset_index(inplace=True, drop=True)
    return all_ratings_for_player.loc[index]

def get_average_team_rating(dataframe, matchdate):
    teamRating: int = 0
    missingPlayers = 0
    for name, series in dataframe.items():
        try:
            teamRating = teamRating + int(get_nearest_rating(date=matchdate, player_api_id=int(series.item()))["overall_rating"])
        except ValueError:
            missingPlayers += 1  
    return teamRating / (11 - missingPlayers)


def get_player_height(player_api_id):
    singleHeight = (player_data['height'].loc[player_data['player_api_id'] == player_api_id]).item()
    if not singleHeight:
        raise ValueError(f"No height-value for provided player_api_id: {player_api_id}")
    else:
        return singleHeight
    

def get_player_weight(player_api_id):
    singleWeight = (player_data['weight'].loc[player_data['player_api_id'] == player_api_id]).item()
    if not singleWeight:
        raise ValueError(f"No weight-value for provided player_api_id: {player_api_id}")
    else:
        return singleWeight


def get_player_age(player_api_id, date):
    birthdate = (player_data['birthday'].loc[player_data['player_api_id'] == player_api_id]).item()
    if not birthdate:
        raise ValueError(f"No age-value for provided player_api_id: {player_api_id}")
    else:
        birthdate = birthdate[:10]
        birthdate = birthdate.replace('-', '') 

        matchYear = str(date)[:4]
        birthYear = str(birthdate)[:4]
        matchMonth = str(date)[4:6]
        birthMonth = str(birthdate)[4:6]

        if int(matchMonth) < int(birthMonth):
            return int(matchYear) - int(birthYear) - 1
        elif int(matchMonth) == int(birthMonth):
            matchDay = str(date)[6:8]
            birthDay = str(birthdate)[6:8]
            if(int(matchDay) < int(birthDay)):
                return int(matchYear) - int(birthYear)
            else:
                return int(matchYear) - int(birthYear) 
        else:
            return int(matchYear) - int(birthYear)


def get_strong_foot(player_api_id):
    
    df_all_feet_player = player_stats_data['preferred_foot'].loc[player_stats_data['player_api_id'] == player_api_id]
    df_all_feet_player.reset_index(inplace=True, drop=True)

    if df_all_feet_player.empty:
        raise ValueError(f"No preferred_foot-value for provided player_api_id: {player_api_id}")
    strongFoot = df_all_feet_player[0]
    
    if str(strongFoot) == 'right':
        return 1
    else:
        return 0


def set_label(dataframe):
    homeGoals = dataframe['home_team_goal'].item()
    awayGoals = dataframe['away_team_goal'].item()

    if homeGoals > awayGoals:
        return 'Win'
    if homeGoals < awayGoals:
        return 'Lose'
    if homeGoals == awayGoals:
        return 'Draw'


def get_last_three_matches(team_api_id, date):
    allHomeGamesOfTeam = match_data[['date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']].loc[match_data['home_team_api_id'] == team_api_id]
    allAwayGamesOfTeam = match_data[['date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']].loc[match_data['away_team_api_id'] == team_api_id]
    allGamesOfTeam = pd.concat([allHomeGamesOfTeam, allAwayGamesOfTeam],axis=0, join='inner')
    allGamesOfTeam = allGamesOfTeam.sort_values(by=['date'])
    for dateIndex, item in allGamesOfTeam['date'].iteritems():
        itemdate: str = item[:10]
        itemdate = itemdate.replace('-', '')
        allGamesOfTeam.loc[dateIndex, 'date'] = int(itemdate)
    allGamesOfTeam = allGamesOfTeam.reset_index()
    indexOfMatch = allGamesOfTeam.loc[allGamesOfTeam['date'] == date].index
    try:
        lastMatch = allGamesOfTeam[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']].loc[(indexOfMatch - 1)]
        lastMatchValue = get_match_value(lastMatch, team_api_id)
    except:
        lastMatchValue = 0
    try:
        secondLastMatch = allGamesOfTeam[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']].loc[(indexOfMatch - 2)]
        secondLastMatchValue = get_match_value(secondLastMatch, team_api_id)
    except:
        secondLastMatchValue = get_match_value(lastMatch, team_api_id)
    try:
        thirdLastMatch = allGamesOfTeam[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']].loc[(indexOfMatch - 3)]
        thirdLastMatchValue = get_match_value(thirdLastMatch, team_api_id)
    except:
        thirdLastMatchValue = get_match_value(lastMatch, team_api_id)
    return (lastMatchValue + secondLastMatchValue + thirdLastMatchValue) 



def get_match_value(dataframe, team_api_id):
    if (dataframe['home_team_api_id'].item() == team_api_id):
        ownGoals = dataframe['home_team_goal'].item()
        otherGoals = dataframe['away_team_goal'].item()
    else:
        ownGoals = dataframe['away_team_goal'].item()
        otherGoals = dataframe['home_team_goal'].item()

    if ownGoals > otherGoals:
        return 3
    if ownGoals < otherGoals:
        return 0
    if ownGoals == otherGoals:
        return 1




df_dropped = df_dropped.assign(average_rating_hometeam=np.NaN)
df_dropped = df_dropped.assign(average_rating_awayteam=np.NaN)
df_dropped = df_dropped.assign(average_height_hometeam=np.NaN)
df_dropped = df_dropped.assign(average_height_awayteam=np.NaN)
df_dropped = df_dropped.assign(average_weight_hometeam=np.NaN)
df_dropped = df_dropped.assign(average_weight_awayteam=np.NaN)
df_dropped = df_dropped.assign(average_age_hometeam=np.NaN)
df_dropped = df_dropped.assign(average_age_awayteam=np.NaN)
df_dropped = df_dropped.assign(right_foot_percentage_hometeam=np.NaN)
df_dropped = df_dropped.assign(right_foot_percentage_awayteam=np.NaN)
df_dropped = df_dropped.assign(label_outcome=np.NaN)
df_dropped = df_dropped.assign(last_three_matches_hometeam=np.NaN)
df_dropped = df_dropped.assign(last_three_matches_awayteam=np.NaN)



for index, match_api_id in df_dropped['match_api_id'].iteritems():
    home_api_id = match_data['home_team_api_id'].loc[match_data['match_api_id'] == match_api_id].item()
    away_api_id = match_data['away_team_api_id'].loc[match_data['match_api_id'] == match_api_id].item()


    df_match = match_data.loc[match_data["match_api_id"] == match_api_id]
    df_match_home_player = df_match[["home_player_1", "home_player_2", "home_player_3", "home_player_4", "home_player_5", "home_player_6", "home_player_7","home_player_8", "home_player_9", "home_player_10", "home_player_11"]]
    df_match_away_player = df_match[["away_player_1", "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6", "away_player_7","away_player_8", "away_player_9", "away_player_10", "away_player_11"]] 

    matchDate: str = df_match["date"].item()
    matchDate = matchDate[:10]
    matchDate = matchDate.replace('-', '')
    matchDateInt: np.int8 = int(matchDate)

    average_team_rating_home = get_average_team_rating(dataframe=df_match_home_player, matchdate=matchDateInt)
    average_team_rating_away = get_average_team_rating(dataframe=df_match_away_player, matchdate=matchDateInt)

    lastThreeMatchesHome = get_last_three_matches(home_api_id, matchDateInt)
    lastThreeMatchesAway = get_last_three_matches(away_api_id, matchDateInt)

    teamHeightHome = 0
    teamWeightHome = 0
    teamAgeHome = 0
    teamRightFootPercentageHome = 0
    missingPlayerHeightHome = 0
    missingPlayerWeightHome = 0
    missingPlayerAgeHome = 0
    missingPlayerStrongFootHome = 0

    for name, series in df_match_home_player.items():
        try:
            teamHeightHome = teamHeightHome + int(get_player_height(player_api_id=int(series.item())))
        except ValueError:
            missingPlayerHeightHome += 1  

        try:
            teamWeightHome = teamWeightHome + int(get_player_weight(player_api_id=int(series.item())))
        except ValueError:
            missingPlayerWeightHome += 1  

        try:
            teamAgeHome = teamAgeHome + int(get_player_age(player_api_id=int(series.item()), date=matchDateInt))
        except ValueError:
            missingPlayerAgeHome += 1  

        try:
            teamRightFootPercentageHome = teamRightFootPercentageHome + int(get_strong_foot(player_api_id=int(series.item())))
        except ValueError:
            missingPlayerStrongFootHome += 1  

    teamHeightHome = teamHeightHome / (11 - missingPlayerHeightHome)
    teamWeightHome = teamWeightHome / (11 - missingPlayerWeightHome)
    teamAgeHome = teamAgeHome / (11 - missingPlayerWeightHome)
    teamRightFootPercentageHome = teamRightFootPercentageHome / (11 - missingPlayerStrongFootHome)


    teamHeightAway = 0
    teamWeightAway = 0
    teamAgeAway = 0
    teamRightFootPercentageAway = 0
    missingPlayerHeightAway = 0
    missingPlayerWeightAway = 0
    missingPlayerAgeAway = 0
    missingPlayerStrongFootAway = 0

    for name, series in df_match_away_player.items():
        try:
            teamHeightAway = teamHeightAway + int(get_player_height(player_api_id=int(series.item())))
        except ValueError:
            missingPlayerHeightAway += 1  

        try:
            teamWeightAway = teamWeightAway + int(get_player_weight(player_api_id=int(series.item())))
        except ValueError:
            missingPlayerWeightAway += 1  

        try:
            teamAgeAway = teamAgeAway + int(get_player_age(player_api_id=int(series.item()), date=matchDateInt))
        except ValueError:
            missingPlayerAgeAway += 1  

        try:
            teamRightFootPercentageAway = teamRightFootPercentageAway + int(get_strong_foot(player_api_id=int(series.item())))
        except ValueError:
            missingPlayerStrongFootAway += 1  

    teamHeightAway = teamHeightAway / (11 - missingPlayerHeightAway)
    teamWeightAway = teamWeightAway / (11 - missingPlayerWeightAway)
    teamAgeAway = teamAgeAway / (11 - missingPlayerWeightAway)
    teamRightFootPercentageAway = teamRightFootPercentageAway / (11 - missingPlayerStrongFootAway)
    label_outcome = set_label(df_match)

    df_dropped.loc[index,'average_rating_hometeam'] = average_team_rating_home
    df_dropped.loc[index,'average_rating_awayteam'] = average_team_rating_away
    df_dropped.loc[index,'average_height_hometeam'] = teamHeightHome
    df_dropped.loc[index,'average_height_awayteam'] = teamHeightAway
    df_dropped.loc[index,'average_weight_hometeam'] = teamWeightHome
    df_dropped.loc[index,'average_weight_awayteam'] = teamWeightAway
    df_dropped.loc[index,'average_age_hometeam'] = teamAgeHome
    df_dropped.loc[index,'average_age_awayteam'] = teamAgeAway
    df_dropped.loc[index,'right_foot_percentage_hometeam'] = teamRightFootPercentageHome
    df_dropped.loc[index,'right_foot_percentage_awayteam'] = teamRightFootPercentageAway
    df_dropped.loc[index,'label_outcome'] = label_outcome
    df_dropped.loc[index,'last_three_matches_hometeam'] = lastThreeMatchesHome
    df_dropped.loc[index,'last_three_matches_awayteam'] = lastThreeMatchesAway

    #print(df_dropped)




#Überprüfung ob jeder Spieler aus Match_data auch in der Spielerdatenbank

In [ ]:
# Select the player columns
player_cols = ['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']
    
# Combine the columns horizontally
combined_players = pd.concat([match_data[col] for col in player_cols])

# Remove duplicates
unique_players = combined_players.drop_duplicates()
print(unique_players[0])

# Check if every entry in the Series is in column A of the DataFrame
mask = unique_players.isin(player_data['player_api_id'])

# Replace the entries not in column A with a specified value
unique_players[~mask] = -1

# find the indices where the value is -1
indices = unique_players.index[unique_players == -1]

# count the number of occurrences of each unique value in the series
counts = unique_players.value_counts()

# print the indices & print the number of occurrences of -1
print(counts[-1])
print(indices)

#Hier muss noch die Zusammenführung der Daten geschehen in ein Dataframe df


In [ ]:
df = pd.DataFrame()

In [ ]:
# Korrelaitonsmatrix erstellt und in corr_matrix gespeichert => enthält Korrleationskoeffizienten zwischen allen möglichen Paaren
corr_matrix = df.corr()

# stack() wandelt Matrix in Series um, bei der jedes Element ein Paar von Spaltennamen und der Korrelationskoeffizient zwischen diesen Spalten ist
# [abs(corr_matrix) > 0.9] reduziert die Werte auf nur Werte mit einem Wert über 0.9
high_corr_pairs = corr_matrix.stack()[abs(corr_matrix.stack()) > 0.9]

# Indizes der Serie werden zu Spaltennamen des Dataframe 
high_corr_pairs = high_corr_pairs.reset_index()

# Spaltennamen des DataFrames umbenannt für bessere Lesbarkeit
high_corr_pairs.columns = ['Column 1', 'Column 2', 'Correlation']

print(high_corr_pairs)

#Chi Squared

In [ ]:
 # erstelle ein Dataframe für die Ergebnis von Chi2
chi2_results_df = pd.DataFrame(columns=['Column 1', 'Column 2', 'Chi-Squared', 'P-Value'])

# iteriert über jede Spalte des Dataframes
for col in df.columns:
    comparison_col = col

    # iteriert über jede Spalte des Dataframes
    for col in df.columns:

        # überspringt den Schritt, wenn die Spalten die gleichen sind
        if col == comparison_col:
            continue

        # erstellt eine Kontingenztabelle
        contingency_table = pd.crosstab(df[comparison_col], df[col])

        # führt chi2 aus und printed das Ergebnis
        chi2, pval, dof, expected = chi2_contingency(contingency_table)
                    #print(f\Chi-squared test for columns '{comparison_col}' and '{col}': chi2={chi2}, pval={pval}\)
        results_df = results_df.append({'Column 1': comparison_col, 'Column 2': col, 'Chi-Squared': chi2, 'P-Value': pval}, ignore_index=True)

    print(results_df)

TODO: Was machen wir mit dem Ergebnis

#Wrapper Methode
Sucht die beste Untergruppe von Features für ein Mdoell, indem eine Modell-basierte Bewertung der Feature-Untergruppen durchführen.

In [ ]:
# Trennung von Features und Zielvariablen
features = df.drop('whoWon', axis=1)
target = df['whoWon']

# Definition des Modells
model = LinearRegression()

# Erstellung des RFE-Objekts mit 10 gewünschten Features
rfe = RFE(model, n_features_to_select=10)

# Anpassung des RFE-Objekts an die Daten
rfe.fit(features, target)

# Ausgabe der ausgewählten Features
selected_features = features.columns[rfe.support_]
print(selected_features)

#Random Forest

In [ ]:
# Trennung von Features und Zielvariablen
features = df.drop('whoWon', axis=1)
target = df['whoWon']

# Aufteilen der Daten in Trainings- und Testdaten
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Definition des Random-Forest-Modells
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Training des Modells auf den Trainingsdaten
rf.fit(features_train, target_train)

# Vorhersage auf den Testdaten
target_pred = rf.predict(features_test)

# Berechnung der Genauigkeit
accuracy = accuracy_score(target_test, target_pred)
print(f"Genauigkeit: {accuracy}")

#Logistische Regression

In [ ]:
# Trennung von Features und Zielvariablen
features = df.drop('whoWon', axis=1)
target = df['whoWon']

# Aufteilen der Daten in Trainings- und Testdaten
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Definition des logistischen Regressionsmodells
lr = LogisticRegression()

# Training des Modells auf den Trainingsdaten
lr.fit(features_train, target_train)

# Vorhersage auf den Testdaten
target_pred = lr.predict(features_test)

# Berechnung der Genauigkeit
accuracy = accuracy_score(target_test, target_pred)
print(f"Genauigkeit: {accuracy}")